In [1]:
import { load } from "dotenv";
const env = await load();

const process = {
    env
}

In [2]:
import { ChatOpenAI } from "@langchain/openai";
import { RunnableSequence } from "@langchain/core/runnables";
import { RunnablePassthrough } from "@langchain/core/runnables";
import { StringOutputParser } from "@langchain/core/output_parsers";
import { BufferMemory } from "langchain/memory";
import { ChatPromptTemplate } from "@langchain/core/prompts";

In [3]:
const chatModel = new ChatOpenAI({
    verbose:true
});
const memory = new BufferMemory();

const TEMPLATE = `
你是一个乐于助人的 ai 助手。尽你所能回答所有问题。

这是跟人类沟通的聊天历史:
{history}

据此回答人类的问题:
{input}
`
const prompt = ChatPromptTemplate.fromTemplate(TEMPLATE);

In [4]:
let tempInput = ""

const chain =  RunnableSequence.from([
    {
        input: new RunnablePassthrough(),
        memoryObject: async (input) => {
            const history = await memory.loadMemoryVariables({
                input
            })
            tempInput = input
            return history
        }
    },
    RunnablePassthrough.assign({
        history: (input) => input.memoryObject.history
    }),
    prompt,
    new RunnablePassthrough({
        func: (input) => console.log(input)
    }),
    chatModel,
    new StringOutputParser(),
    new RunnablePassthrough({
        func: async (output) => {
            await memory.saveContext({
                input: tempInput
            }, {
                output
            })
        }
    }),
]);

In [5]:
await chain.invoke("你好, 我叫小明")

ChatPromptValue {
  lc_serializable: true,
  lc_kwargs: {
    messages: [
      HumanMessage {
        lc_serializable: true,
        lc_kwargs: {
          content: "\n你是一个乐于助人的 ai 助手。尽你所能回答所有问题。\n\n这是跟人类沟通的聊天历史:\n\n\n据此回答人类的问题:\n你好, 我叫小明\n",
          additional_kwargs: {},
          response_metadata: {}
        },
        lc_namespace: [ "langchain_core", "messages" ],
        content: "\n你是一个乐于助人的 ai 助手。尽你所能回答所有问题。\n\n这是跟人类沟通的聊天历史:\n\n\n据此回答人类的问题:\n你好, 我叫小明\n",
        name: undefined,
        additional_kwargs: {},
        response_metadata: {}
      }
    ]
  },
  lc_namespace: [ "langchain_core", "prompt_values" ],
  messages: [
    HumanMessage {
      lc_serializable: true,
      lc_kwargs: {
        content: "\n你是一个乐于助人的 ai 助手。尽你所能回答所有问题。\n\n这是跟人类沟通的聊天历史:\n\n\n据此回答人类的问题:\n你好, 我叫小明\n",
        additional_kwargs: {},
        response_metadata: {}
      },
      lc_namespace: [ "langchain_core", "messages" ],
      content: "\n你是一个乐于助人的 ai 助手。尽你所能回答所有问题。\n\n这是跟人类沟通的聊天历史:\n\n\n据

"你好小明！有什么问题我可以帮助你解答吗？"

In [6]:
await chain.invoke("我叫什么？")

ChatPromptValue {
  lc_serializable: true,
  lc_kwargs: {
    messages: [
      HumanMessage {
        lc_serializable: true,
        lc_kwargs: {
          content: "\n" +
            "你是一个乐于助人的 ai 助手。尽你所能回答所有问题。\n" +
            "\n" +
            "这是跟人类沟通的聊天历史:\n" +
            "Human: 你好, 我叫小明\n" +
            "AI: 你好小明！有什么问题我可以帮助你解答吗？\n" +
            "\n" +
            "据此回答人类的问题:\n" +
            "我叫什么？\n",
          additional_kwargs: {},
          response_metadata: {}
        },
        lc_namespace: [ "langchain_core", "messages" ],
        content: "\n" +
          "你是一个乐于助人的 ai 助手。尽你所能回答所有问题。\n" +
          "\n" +
          "这是跟人类沟通的聊天历史:\n" +
          "Human: 你好, 我叫小明\n" +
          "AI: 你好小明！有什么问题我可以帮助你解答吗？\n" +
          "\n" +
          "据此回答人类的问题:\n" +
          "我叫什么？\n",
        name: undefined,
        additional_kwargs: {},
        response_metadata: {}
      }
    ]
  },
  lc_namespace: [ "langchain_core", "prompt_values" ],
  messages: [
    HumanMessage {
    

"AI: 你叫小明。"

## 实现自定义的 chat history

In [7]:
import { JSONChatHistory } from "./JSONChatHistory/index.ts"
import { AIMessage, HumanMessage } from "@langchain/core/messages";

const history = new JSONChatHistory({
    dir: "chat_data",
    sessionId: "test"
})


await history.addMessages([
  new HumanMessage("Hi, 我叫小明"),
  new AIMessage("你好"),
]);

const messages = await history.getMessages();
console.log(messages)

[
  HumanMessage {
    lc_serializable: true,
    lc_kwargs: {
      content: "Hi, 我叫小明",
      additional_kwargs: {},
      response_metadata: {}
    },
    lc_namespace: [ "langchain_core", "messages" ],
    content: "Hi, 我叫小明",
    name: undefined,
    additional_kwargs: {},
    response_metadata: {}
  },
  AIMessage {
    lc_serializable: true,
    lc_kwargs: { content: "你好", additional_kwargs: {}, response_metadata: {} },
    lc_namespace: [ "langchain_core", "messages" ],
    content: "你好",
    name: undefined,
    additional_kwargs: {},
    response_metadata: {}
  },
  HumanMessage {
    lc_serializable: true,
    lc_kwargs: { content: "我叫什么？", additional_kwargs: {}, response_metadata: {} },
    lc_namespace: [ "langchain_core", "messages" ],
    content: "我叫什么？",
    name: undefined,
    additional_kwargs: {},
    response_metadata: {}
  },
  AIMessage {
    lc_serializable: true,
    lc_kwargs: { content: "你叫小明。", additional_kwargs: {}, response_metadata: {} },
    lc_namespac

In [8]:
import { ChatOpenAI } from "@langchain/openai";
import { BufferMemory } from "langchain/memory";
import { ConversationChain } from "langchain/chains";


const chatModel = new ChatOpenAI();
const memory = new BufferMemory({
    chatHistory: history
});
const chain = new ConversationChain({ llm: chatModel, memory: memory });
const res1 = await chain.call({ input: "我叫什么？" });

In [9]:
res1

{ response: "你叫小明。" }

In [10]:
const messages = await history.getMessages()

In [11]:
messages

[
  HumanMessage {
    lc_serializable: true,
    lc_kwargs: {
      content: "Hi, 我叫小明",
      additional_kwargs: {},
      response_metadata: {}
    },
    lc_namespace: [ "langchain_core", "messages" ],
    content: "Hi, 我叫小明",
    name: undefined,
    additional_kwargs: {},
    response_metadata: {}
  },
  AIMessage {
    lc_serializable: true,
    lc_kwargs: { content: "你好", additional_kwargs: {}, response_metadata: {} },
    lc_namespace: [ "langchain_core", "messages" ],
    content: "你好",
    name: undefined,
    additional_kwargs: {},
    response_metadata: {}
  },
  HumanMessage {
    lc_serializable: true,
    lc_kwargs: { content: "我叫什么？", additional_kwargs: {}, response_metadata: {} },
    lc_namespace: [ "langchain_core", "messages" ],
    content: "我叫什么？",
    name: undefined,
    additional_kwargs: {},
    response_metadata: {}
  },
  AIMessage {
    lc_serializable: true,
    lc_kwargs: { content: "你叫小明。", additional_kwargs: {}, response_metadata: {} },
    lc_namespac